In [4]:
import json
import pandas as pd
import statistics as stats
import os

parsedData = []

jsonFilePath = 'searchResults-βαπτίζω.json'
with open(jsonFilePath, 'r') as json_file:
    data = json.load(json_file)

def reformatData(item):
    newItem = item.copy()
    for key in ['lemma', 'strong', 'sourceText', 'targetText']:
        value = item[key]
        count = len(value.split())
        newItem[key + 'Count'] = count

    del newItem['refs']
    del newItem['count']
    return newItem

for i in range(len(data)):
    item = data[i]
    refs = item['refs']
    newItem = reformatData(item)

    for j in range(len(refs)):
        ref = refs[j]
        newItem_ = newItem.copy()
        newItem_['ref'] = ref
        targetsPosStr = newItem['targetsPos'][j]
        targetsPos = targetsPosStr.split()
        positions = []
        for pos in targetsPos:
            positions.append(int(pos))
        newItem_['targetsPos'] = targetsPosStr

        deltas = []
        for k in range(len(positions) - 1):
            delta = abs(positions[k] - positions[k + 1])
            deltas.append(delta)
        # newItem_['deltas'] = deltas
        
        if len(deltas) == 0:
            deltaAve = 0
            deltaVar = 0
        else:
            deltaAve = stats.mean(deltas)
            if len(deltas) == 1:
                deltaVar = 0
            else:
                deltaVar = stats.variance(deltas)

        newItem_['deltaAve'] = deltaAve
        newItem_['deltaVar'] = deltaVar
        
        parsedData.append(newItem_)


base_path, file_name = os.path.split(jsonFilePath)
base_name, ext = os.path.splitext(file_name)
outputFileName = 'parsedAlignments-' + base_name + ext
outputFilePath = os.path.join(base_path, outputFileName)
with open(outputFilePath, 'w') as json_file:
    json.dump(parsedData, json_file)
    
df = pd.DataFrame.from_dict(parsedData)

print(len(parsedData), "rows found")

df

75 rows found


,morph,lemma,sourceText,strong,targetText,targetsPos,lemmaCount,strongCount,sourceTextCount,targetTextCount,ref,deltaAve,deltaVar
0,"Gr,V,IIP3,,P,",βαπτίζω,ἐβαπτίζοντο,G09070,being baptized,1 2,1,1,1,2,mat 3:6,1.000000,0.000000
1,"Gr,V,IPA1,,S,",βαπτίζω,βαπτίζω,G09070,baptize,1,1,1,1,1,mat 3:11,0.000000,0.000000
2,"Gr,V,IPA1,,S,",βαπτίζω,βαπτίζω,G09070,baptize,8,1,1,1,1,luk 3:16,0.000000,0.000000
3,"Gr,V,IPA1,,S,",βαπτίζω,βαπτίζω,G09070,baptize,5,1,1,1,1,jhn 1:26,0.000000,0.000000
4,"Gr,V,IFA3,,S,",βαπτίζω,βαπτίσει,G09070,will baptize,29 30,1,1,1,2,mat 3:11,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,"Gr,V,IPP3,,P,",βαπτίζω,βαπτίζονται,G09070,are they baptized,21 22 23,1,1,1,3,1co 15:29,1.000000,0.000000
71,"Gr,V,IAP2,,P,",βαπτίζω,ἐβαπτίσθητε,G09070,have been baptized,4 5 6,1,1,1,3,gal 3:27,1.000000,0.000000
72,"Gr,V,PEP,NMP, Gr,V,IIA3,,P,",βαπτίζω ὑπάρχω,βεβαπτισμένοι ὑπῆρχον,G09070 G52250,they had been baptized,11 12 14 15,2,2,2,4,act 8:16,1.333333,0.333333
73,"Gr,RD,,,,GNS, Gr,V,NAP,,,,,",ὁ βαπτίζω,τοῦ βαπτισθῆναι,G35880 G09070,to be baptized,10 11 12,2,2,2,3,mat 3:13,1.000000,0.000000
